In [1]:
import psycopg2
from psycopg2 import sql
from pathlib import Path


Connection configuration

In [2]:
username = 'postgres'
password = 'kasztanP7r.U1'

host = 'localhost'
database = 'airlines'
port = '5432'

defining the con and cursor variables

In [3]:
con = psycopg2.connect(dbname=database, user=username, password=password, host=host, port=port)
cursor = con.cursor()

In [4]:
# Creating a connection
try:
    con = psycopg2.connect(dbname=database, user=username, password=password, host=host, port=port)
    cursor = con.cursor()
    print("Połączenie z bazą danych zostało ustanowione.")
except psycopg2.Error as e:
    print("Błąd podczas nawiązywania połączenia z bazą danych:", e)

Połączenie z bazą danych zostało ustanowione.


Loading the reporting.sql file

In [5]:
# Path to the database schema file
schema_file_path = Path(r'C:\Users\eweli\Downloads\Pliki_do_projektu_końcowego\sql\reporting.sql')

# Loading the contents of the database schema file
try:
    with open(schema_file_path, 'r') as file:
        schema_sql = file.read()
    print("Plik reporting.sql został wczytany pomyślnie.")
except FileNotFoundError:
    print("Nie można znaleźć pliku reporting.sql w podanej lokalizacji.")

Plik reporting.sql został wczytany pomyślnie.


In [6]:
# Contents of the reporting.sql file
schema_content = """
CREATE SCHEMA IF NOT EXISTS reporting;

CREATE OR REPLACE VIEW reporting.flight AS
SELECT 
    *,
    CASE WHEN dep_delay_new > 0 THEN 1 ELSE 0 END AS is_delayed
FROM 
    public.flight
WHERE 
    cancelled = 0;

CREATE OR REPLACE VIEW reporting.top_reliability_roads AS
SELECT 
    f.origin_airport_id,
    a1.name AS origin_airport_name,
    f.dest_airport_id,
    a2.name AS dest_airport_name,
    f.year,
    COUNT(*) AS cnt,
    (SUM(CASE WHEN f.arr_delay_new > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS reliability,
    DENSE_RANK() OVER (PARTITION BY f.origin_airport_id, f.dest_airport_id ORDER BY (SUM(CASE WHEN f.arr_delay_new > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) DESC) AS nb
FROM 
    public.flight f
JOIN 
    public.airport_list a1 ON f.origin_airport_id = a1.origin_airport_id
JOIN 
    public.airport_list a2 ON f.dest_airport_id = a2.origin_airport_id
GROUP BY 
    f.origin_airport_id, a1.name, f.dest_airport_id, a2.name, f.year
HAVING 
    COUNT(*) > 10000;


CREATE OR REPLACE VIEW reporting.year_to_year_comparision AS
SELECT 
    year,
    month,
    COUNT(*) AS flights_amount,
    (SUM(CASE WHEN arr_delay_new > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS reliability
FROM 
    public.flight
GROUP BY 
    year, month;

CREATE OR REPLACE VIEW reporting.day_to_day_comparision AS
SELECT 
    year,
    day_of_week,
    COUNT(*) AS flights_amount
FROM 
    public.flight
GROUP BY 
    year, day_of_week;

CREATE OR REPLACE VIEW reporting.day_by_day_reliability AS
SELECT 
    TO_DATE(LPAD(year::text, 4, '0') || '-' || LPAD(month::text, 2, '0') || '-' || LPAD(day_of_month::text, 2, '0'), 'YYYY-MM-DD') AS date,
    (SUM(CASE WHEN arr_delay_new > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS reliability
FROM 
    public.flight
GROUP BY 
    year, month, day_of_month;

"""

# Division of content into queries
queries = schema_content.split(';')

# Removing whitespace from queries
queries = [query.strip() for query in queries if query.strip()]

# View split queries
for query in queries:
    print(query)
    print('-' * 50)


CREATE SCHEMA IF NOT EXISTS reporting
--------------------------------------------------
CREATE OR REPLACE VIEW reporting.flight AS
SELECT 
    *,
    CASE WHEN dep_delay_new > 0 THEN 1 ELSE 0 END AS is_delayed
FROM 
    public.flight
WHERE 
    cancelled = 0
--------------------------------------------------
CREATE OR REPLACE VIEW reporting.top_reliability_roads AS
SELECT 
    f.origin_airport_id,
    a1.name AS origin_airport_name,
    f.dest_airport_id,
    a2.name AS dest_airport_name,
    f.year,
    COUNT(*) AS cnt,
    (SUM(CASE WHEN f.arr_delay_new > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS reliability,
    DENSE_RANK() OVER (PARTITION BY f.origin_airport_id, f.dest_airport_id ORDER BY (SUM(CASE WHEN f.arr_delay_new > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) DESC) AS nb
FROM 
    public.flight f
JOIN 
    public.airport_list a1 ON f.origin_airport_id = a1.origin_airport_id
JOIN 
    public.airport_list a2 ON f.dest_airport_id = a2.origin_airport_id
GROUP BY 
    f.origin_air

In [7]:
# executing each query to initialize the database structure
for query in queries:
    if query.strip():  # Sprawdzenie, czy kwerenda nie jest pusta
        try:
            cursor.execute(query)
            print("Kwerenda została wykonana pomyślnie.")
        except psycopg2.Error as e:
            print("Błąd podczas wykonywania kwerendy:", e)

Kwerenda została wykonana pomyślnie.
Kwerenda została wykonana pomyślnie.
Kwerenda została wykonana pomyślnie.
Kwerenda została wykonana pomyślnie.
Kwerenda została wykonana pomyślnie.
Kwerenda została wykonana pomyślnie.


Approval of all operations on the database, i.e. creation of views

In [8]:
# Approval of changes
con.commit()


Verification

In [9]:
def check_if_table_exists(table_name):
    msg = f"Sprawdzam czy istnieje tabela {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    # jeżeli tabela nie istnieje, ten krok zwróci wyjątek
    cursor.execute(query)
    print('OK!')

In [10]:
views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

In [11]:
for view in views_to_test:
    check_if_table_exists(view)

Sprawdzam czy istnieje tabela reporting.flight
OK!
Sprawdzam czy istnieje tabela reporting.top_reliability_roads
OK!
Sprawdzam czy istnieje tabela reporting.year_to_year_comparision
OK!
Sprawdzam czy istnieje tabela reporting.day_to_day_comparision
OK!
Sprawdzam czy istnieje tabela reporting.day_by_day_reliability
OK!


In [12]:
con.close()
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego zadania."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego zadania.
